In [129]:
import numpy as np
import pandas as pd
import unittest
import ipywidgets as widgets
import matplotlib.pyplot as plt
    

In [153]:

class Laguerre:
    def __init__(self, beta=2, sigma=4):
        self._beta = beta
        self._sigma = sigma
        
    
    @property
    def sigma(self):
        return self._sigma
    
    @sigma.setter
    def sigma(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError("sigma must be a number")
        elif self.beta > value:
            raise ValueError("sigma must be greater than beta")
        self._sigma = value
    
    @property
    def beta(self):
        return self._beta
    
    @beta.setter
    def beta(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError("beta must be a number")
        elif self.sigma < value:
            raise ValueError("beta must be less than sigma")
        self._beta = value

    def laguerre(self, t, n):
        if self.beta > self.sigma or self.beta < 0:
            raise ValueError("Wrong beta and omega")
        
        l_0 = np.sqrt(self.sigma) * (np.exp(-self.beta * t / 2))
        l_1 = np.sqrt(self.sigma) * (1 - self.sigma * t) * (np.exp(-self.beta * t / 2))

        if n == 0:
            return l_0
        elif n == 1:
            return l_1
        
        l_next = (3 - t * self.sigma) / 2 * l_1 - 0.5 * l_0
        for j in range(3, n+1):
            l_0 = l_1
            l_1 = l_next
            l_next = (2 * j - 1 - t * self.sigma) / j * l_1 - (j - 1) / j * l_0
        return l_next
    
    def tabulate_laguerre(self, n, T):
        steps = np.linspace(0, T, 100)
        res = [self.laguerre(t, n) for t in steps]
        res_df = pd.DataFrame({"n": steps, "result": res})
        return res_df
    
    def experiment(self, eps=0.001):
        N = 20
        t = 0
        while True:
            t += 0.001
            res = []
            flag = True
            for i in range(N):
                val = abs(self.laguerre(t, i))
                if val > eps:
                    flag = False
                    break
                res.append(val)
            if flag:
                return t, pd.DataFrame({"n": range(N), "res": res})
    
    def rects_integral(self, f, T, N=1000):
        step = T / N
        integral = 0
        for i in range(N):
            t = i * step
            integral += f(t) * step
        return integral
    
    
    
    def laguerre_transformation(self, f, n, T):
        alpha = self.beta - self.sigma
        integrand = lambda t: f(t) * self.laguerre(t, n) * np.exp(-t * alpha)
        result = self.rects_integral(integrand, T)
        return result
    
    def tab_transformation(self, f, T, N=20):
        t = range(1, N+1)
        results = [self.laguerre_transformation(f, n, T) for n in t]
        return results

    def reverse_laguerre_transformation(self, lst, t):
        return sum([lst[i] * self.laguerre(t, i) for i in range(len(lst))])
    
    def plot_laguerre(self, T, N):
        fig = plt.figure(figsize=(10, 10))
        ax = fig.gca()
        for n in range(N+1):
            values = self.tabulate_laguerre(n, T)
            ax.plot(values['n'], values['result'], label=f"n={n}")
        
        ax.set_xlabel("t")
        ax.set_ylabel("l(t)")
        fig.legend(loc='lower center', ncol=4)
        plt.show()
    
    def plot_transformation(self, f, n, T):
        fig = plt.figure(figsize=(10, 10))
        ax = fig.gca()
        values = self.tab_transformation(f, T, n)
        ax.bar(range(1, n+1), values)

        ax.set_xlabel("n")
        ax.set_ylabel("f_n")
        ax.set_title("Laguerre Transformation")
        plt.axhline(0, color='black')
        plt.show()
        
    def plot_transformations(self, f, n, T, t1=0, t2=2*np.pi):
        transform_values = self.tab_transformation(f, T, n)
        reversed_transform_values = [self.reverse_laguerre_transformation(transform_values, t) for t in np.linspace(t1, t2, 1000)]

        fig, axs = plt.subplots(2, 1, figsize=(10, 10))

        axs[0].bar(range(1, n+1), transform_values, alpha=0.7, edgecolor='black')
        axs[0].set_xlabel("n")
        axs[0].set_ylabel("f_n")
        axs[0].set_title("Transformation coefs")
        axs[0].set_xticks(range(1, n+1))
        axs[0].axhline(0, color='black')

        axs[1].plot(np.linspace(t1, t2, 1000), reversed_transform_values, alpha=0.7, linewidth=2.0)
        axs[1].set_xlabel("t")
        axs[1].set_ylabel("f(t)")
        axs[1].set_title("Reversed transformation")

        plt.tight_layout()
        plt.show()
    

In [154]:
class TestLaguerre(unittest.TestCase):
    def setUp(self):
        self.laguerre_instance = Laguerre(beta=1, sigma=5)

    def test_laguerre(self):
        self.assertAlmostEqual(self.laguerre_instance.laguerre(0, 0), 2.23606797749979)
        self.assertAlmostEqual(self.laguerre_instance.laguerre(1, 3), 3.6166500948139766)

    def test_tabulate_laguerre(self):
        tabulated_values = self.laguerre_instance.tabulate_laguerre(2, 10)
        self.assertEqual(len(tabulated_values), 100)
        self.assertIsInstance(tabulated_values, pd.DataFrame)

    def test_experiment(self):
        t, result_df = self.laguerre_instance.experiment(eps=0.03)
        self.assertAlmostEqual(t, 189.64000000054145)
        self.assertEqual(len(result_df), 20)

    def test_laguerre_transformation(self):
        def f(t):
            if t >= 0 and t <= 2*np.pi:
                return np.sin(t-np.pi/2) + 1
            else:
                return 0
        transformation_result = self.laguerre_instance.laguerre_transformation(f, 5, 10)
        self.assertAlmostEqual(transformation_result, -7833210664009.94 )

    def test_reverse_laguerre_transformation(self):
        def f(t):
            if t >= 0 and t <= 2*np.pi:
                return np.sin(t-np.pi/2) + 1
            else:
                return 0
        coefficients = self.laguerre_instance.tab_transformation(f, 100)
        reversed_transformation_result = self.laguerre_instance.reverse_laguerre_transformation(coefficients, 2)
        self.assertAlmostEqual(reversed_transformation_result, -78419345421126.88 )



In [155]:
unittest.main(argv=[''], exit=False)


.....
----------------------------------------------------------------------
Ran 5 tests in 20.347s

OK


In [156]:
lag = Laguerre()

widgets.interact(lag.laguerre, t=(0.0, 5.0, 0.1), n=(0, 10, 1))


interactive(children=(FloatSlider(value=2.5, description='t', max=5.0), IntSlider(value=5, description='n', ma…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [157]:
widgets.interact(lag.tabulate_laguerre, n=(0, 10, 1), T=(1, 10, 1))


interactive(children=(IntSlider(value=5, description='n', max=10), IntSlider(value=5, description='T', max=10,…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [158]:
widgets.interact(lag.experiment, eps=(0, 0.1, 0.01))


interactive(children=(FloatSlider(value=0.001, description='eps', max=0.1, step=0.01), Output()), _dom_classes…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [159]:
def f(t):
    if 0 <= t <= 2 * np.pi:
        return np.sin(t - np.pi / 2) + 1
    elif t > 2 * np.pi:
        return 0
    


In [160]:
widgets.interact(lag.tab_transformation, f=widgets.fixed(f), T=(1,100,1), N=(1,30,1))


interactive(children=(IntSlider(value=50, description='T', min=1), IntSlider(value=20, description='N', max=30…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [161]:
lst = lag.tab_transformation(f, 100)
widgets.interact(lag.reverse_laguerre_transformation, lst=widgets.fixed(lst), t=(0,10,1))

interactive(children=(IntSlider(value=5, description='t', max=10), Output()), _dom_classes=('widget-interact',…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [162]:
widgets.interact(lag.plot_laguerre, T=(0.0, 20.0, 0.25), N=(0, 30, 1))


interactive(children=(FloatSlider(value=10.0, description='T', max=20.0, step=0.25), IntSlider(value=15, descr…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [163]:
widgets.interact(lag.plot_transformation, f=widgets.fixed(f), T=(0.0, 20.0, 0.25), n=(0, 30, 1))

interactive(children=(IntSlider(value=15, description='n', max=30), FloatSlider(value=10.0, description='T', m…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [165]:
widgets.interact(lag.plot_transformations, f=widgets.fixed(f),  T=(0.0, 20.0, 0.25), n=(0, 30, 1), t1=(-5*np.pi, 5*np.pi, np.pi), t2=(-5*np.pi, 5*np.pi, np.pi))


interactive(children=(IntSlider(value=15, description='n', max=30), FloatSlider(value=10.0, description='T', m…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>